In [28]:
import json
import math

In [16]:
f = open("all-songs.txt", "r")
BLACKLIST_SONGS = json.loads(f.read())
f.close()

f = open("song_analyses.txt", "r")
BLACKLIST_ANALYSIS = json.loads(f.read())
f.close()

f = open("original_songs.txt", "r")
METALLICA_SONGS = json.loads(f.read())
f.close()

f = open("original_song_analyses.txt", "r")
METALLICA_ANALYSIS = json.loads(f.read())
f.close()

In [12]:
# Create dictionary mapping song name to Spotify ID on Metallica (1991)

SONG_IDS = {}
for song in METALLICA_SONGS:
    remove_remastered = len(song["name"]) - len(" (Remastered)")
    processed_name = song["name"][0:remove_remastered]
    SONG_IDS[processed_name] = song["id"]

In [23]:
# Match each cover on the Blacklist to its original

for song in BLACKLIST_SONGS:
    if song["name"] in SONG_IDS:
        song["originals"] = [SONG_IDS[song["name"]]]
    else:
        if song["name"] == "Sad But True (Live)":
            song["originals"] = [SONG_IDS["Sad But True"]]
        elif song["name"] == "Don't Tread on Else Matters":
            song["originals"] = [SONG_IDS["Don't Tread on Me"], SONG_IDS["Nothing Else Matters"]]

f = open("all-songs.txt", "w")
f.write(json.dumps(BLACKLIST_SONGS, indent=4))
f.close()

In [44]:
# Compare features of original and cover

analysis_features = ["danceability", "energy", "speechiness", "acousticness", "instrumentalness", "liveness", "valence"]
other_features = ["key", "mode", "tempo", "loudness"]

csv_header = 'Title,Artists,Danceability,dDanceability,Energy,dEnergy,Speechiness,dSpeechiness,Acousticness,dAcousticness,Instrumentalness,dInstrumentalness,Liveness,dLiveness,Valence,dValence,Overall Difference,Key,dKey,Mode,dMode,Tempo,dTempo,Loudness, dLoudness\n'
csv_body = ""

for song in BLACKLIST_SONGS:

    csv = ""
    
    # This is just for me to look at
    song_artists = ""
    for artist in song['artists']:
        song_artists += artist['name'] + ", "
    song_artists = song_artists[0:len(song_artists) - 2]
    #song_display_name = song['name'] + " by " + song_artists
    csv += f"{song['name']},\"{song_artists}\","
    
    analysis = BLACKLIST_ANALYSIS[song['id']]

    # Most songs are a cover of one song so for now I'm going to only compare it with the first "original"
    original_analysis = METALLICA_ANALYSIS[song['originals'][0]]

    overall_difference = 0
    for feature in analysis_features:
        original_feature = original_analysis[feature]
        cover_feature = analysis[feature]
        difference = cover_feature - original_feature
        #print(f"{feature.upper()}:\tOriginal={original_feature},\tCover={cover_feature},\tDifference={difference}")
        overall_difference += difference ** 2
        csv += f"{cover_feature},{difference},"
    overall_difference = math.sqrt(overall_difference)
    #print("OVERALL DIFFERENCE: " + str(overall_difference))

    csv += str(overall_difference) + ","

    for feature in other_features:
        original_feature = original_analysis[feature]
        cover_feature = analysis[feature]
        difference = cover_feature - original_feature
        csv += f"{cover_feature},{difference},"
    csv_body += csv + "\n"

f = open("feature_differences.csv", "w")
f.write(csv_header + csv_body)
f.close()